In [115]:
#!pip install twython

In [116]:
import re
import string
import csv
import nltk
import unicodedata

from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.corpus import stopwords
from nltk.sentiment.util import *
from nltk.metrics import ConfusionMatrix
from sklearn.model_selection import train_test_split

import sqlite3
from sqlite3 import Error

In [117]:
def processSentence(sentence):
    # process the tweets

    #Convert to lower case
    sentence = sentence.lower()
    #Convert www.* or https?://* to URL
    sentence = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',sentence)
    #Convert @username to AT_USER
    sentence = re.sub('@[^\s]+','AT_USER',sentence)
    #Remove additional white spaces
    sentence = re.sub('[\s]+', ' ', sentence)
    #Replace #word with word
    sentence = re.sub(r'#([^\s]+)', r'\1', sentence)
    #trimd
    sentence = sentence.strip('\'"')
    #retira açentos e caracteres especiais
    sentence = str(unicodedata.normalize('NFKD', sentence).encode('ascii','ignore'),'utf8')
    #************************* 
    sentence = sentence.replace("tmb", "tambem")
    sentence = sentence.replace("tbm", "tambem")
    sentence = sentence.replace("vc", "voce")
    sentence = sentence.replace("sdd", "saudade")
    sentence = sentence.replace("sdds", "saudade")
    sentence = sentence.replace("dnv", "novamente")
    sentence = sentence.replace("pfvr", "por favor")
    sentence = sentence.replace("/", " ")
    sentence = sentence.replace("numca", "nunca")
    return sentence

In [118]:
#initialize stopWords
stopWords = []

#start replaceTwoOrMore
def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    #return pattern.sub(r"\1\1", s)
    return pattern.sub(r"\1", s)
#end

#start getStopWordList
def getStopWordList(stopWordListFileName):
    #read the stopwords file and build a list
    stopWords = []
    
    if 1==1:
        stopWords = nltk.corpus.stopwords.words('portuguese')
    else:
        fp = open(stopWordListFileName, 'r')
        line = fp.readline()
        while line:
            word = line.strip()
            stopWords.append(word)
            line = fp.readline()
        fp.close()
        
    #****************************************
    stopWords.append('AT_USER')
    stopWords.append('URL')
    stopWords.append('url')
    stopWords.append('pra')
    stopWords.append('q')
    stopWords.append('at_user')
    stopWords.append('poder')
    stopWords.append('profunda')
    stopWords.append('poder')
    stopWords.append('voce')
    stopWords.append('to')    
    #*****************************************
    stopWords.remove('nem')
    #stopWords.remove('cabeca')
    #stopWords.remove('cabeça')
    #*****************************************
            
    return stopWords
#end

In [119]:
#start getfeatureVector
def getFeatureVector(sentence,stopWords):
    featureVector = []
    #split tweet into words
    words = sentence.split()
    for w in words:        
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        if len(w)>0:
            if 1!=1:
                #print(len(w))
                aux = w
                stemmer = nltk.stem.SnowballStemmer('portuguese')
                w = stemmer.stem(w)
                #if w=='indo':
                    #print(aux)
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stopWords or val is None):
            continue
        else:
            featureVector.append(w.lower())
        
    return featureVector
#end


In [120]:
def extract_features(sentence):
    sentence_words = set(sentence)
    features = {}
    for word in featureList:
        features['contains(%s)' % word] = (word in sentence_words)
    return features
#end

In [121]:
#Read the sentence one by one and process it
#inpTweets = csv.reader(open('BaseReLi2.csv', 'r'), delimiter=',')

conn = sqlite3.connect('tweets.sqlite')
cur = conn.cursor()
cur.execute("SELECT * FROM tweet where tweet_id == '0'")
inpSentence = cur.fetchall()
    
stopWords = getStopWordList('portuguese')
featureList = []
featureVectorPositive=[]
featureVectorNonPositive=[]

# Get tweet words
cont_neg=0
cont_pos=0
cont_neu=0
cont_max=3000
item_vec_max = 1

for row in inpSentence:
    
    if len(row)==4:
        if row[3] == 1 and cont_pos <= cont_max:
            cont_pos +=1
            sentiment = row[3]
            sentence = row[2]
            processedSentence = processSentence(sentence)
            featureVector = getFeatureVector(processedSentence, stopWords)
            if len(featureVector)>item_vec_max:
                featureVectorPositive.append(" ".join(featureVector))
                1==1
            else:
                #print(featureVector, sentiment)
                1==1

        elif row[3] == -1 and cont_neg <= cont_max:
            cont_neg +=1
            sentiment = row[3]
            sentence = row[2]
            processedSentence = processSentence(sentence)
            featureVector = getFeatureVector(processedSentence, stopWords)
            if len(featureVector)>item_vec_max:
                featureVectorNonPositive.append(" ".join(featureVector))
                1==1
            else:
                #print(featureVector, sentiment)
                1==1

        elif row[3] == 0 and cont_neu <= cont_max:
            cont_neu +=1
            sentiment = row[3]
            sentence = row[2]
            processedSentence = processSentence(sentence)
            featureVector = getFeatureVector(processedSentence, stopWords)
            if len(featureVector)>item_vec_max:
                featureVectorPositive.append(" ".join(featureVector))
                1==1
            else:
                #print(featureVector, sentiment)
                1==1
#end loop

In [122]:
def features(sentence):
    words = sentence.lower().split()
    d={}
    for w in words:
        if len(w)>1:
            d['contains(%s)' % w]=True
        
    return d

In [123]:
featureVectorNonPositive = list(map(features, featureVectorNonPositive))

In [124]:
featureVectorPositive = list(map(features, featureVectorPositive))

In [125]:
FVP_train ,FVP_test = train_test_split(featureVectorPositive,random_state=24061976,shuffle=True,test_size=0.10)
FVnP_train ,FVnP_test = train_test_split(featureVectorNonPositive,random_state=24061976,shuffle=True,test_size=0.10)

NAIVE BAYES CLASSIFIER

In [126]:
from nltk.classify import PositiveNaiveBayesClassifier
classifier = PositiveNaiveBayesClassifier.train(featureVectorPositive,featureVectorNonPositive)

In [129]:
test = '@henriaradia :( eu acho tão mágica'
processedTestTweet = processSentence(test)
print(processedTestTweet)
test = " ".join((getFeatureVector(processedTestTweet,stopWords)))
print(test)
print(features(test))
dic = features(test)
if len(dic)>1:
    print(classifier.classify(dic))
else:
    print(None)

AT_USER :( eu acho tao magica
acho tao magica
{'contains(acho)': True, 'contains(tao)': True, 'contains(magica)': True}
False


In [130]:
classifier.prob_classify(dic).prob(False)

0.8756404169511122

In [82]:
print(classifier.show_most_informative_features(10000))

Most Informative Features
      contains(leitores) = True             True : False  =    830.8 : 1.0
   contains(emocionante) = True             True : False  =    830.8 : 1.0
       contains(centeio) = True             True : False  =    830.8 : 1.0
         contains(lugar) = True             True : False  =    117.1 : 1.0
         contains(fraca) = True            False : True   =     68.8 : 1.0
   contains(previsiveis) = True            False : True   =     66.3 : 1.0
      contains(monotono) = True            False : True   =     54.1 : 1.0
    contains(repetitiva) = True            False : True   =     54.1 : 1.0
           contains(sal) = True            False : True   =     54.1 : 1.0
      contains(ridiculo) = True            False : True   =     54.1 : 1.0
    contains(entediante) = True            False : True   =     46.7 : 1.0
         contains(fraco) = True            False : True   =     43.6 : 1.0
           contains(sol) = True            False : True   =     43.6 : 1.0

       contains(celebre) = True            False : True   =      5.1 : 1.0
      contains(esmiucar) = True            False : True   =      5.1 : 1.0
    contains(brecacrime) = True            False : True   =      5.1 : 1.0
    contains(grandiosos) = True            False : True   =      5.1 : 1.0
    contains(contribuem) = True            False : True   =      5.1 : 1.0
      contains(pergunto) = True            False : True   =      5.1 : 1.0
     contains(adquirida) = True            False : True   =      5.1 : 1.0
     contains(olhasemos) = True            False : True   =      5.1 : 1.0
    contains(enfrentara) = True            False : True   =      5.1 : 1.0
      contains(imprimia) = True            False : True   =      5.1 : 1.0
     contains(cativaram) = True            False : True   =      5.1 : 1.0
     contains(desgostei) = True            False : True   =      5.1 : 1.0
           contains(vis) = True            False : True   =      5.1 : 1.0
        contains(esgota) 

      contains(enganava) = True            False : True   =      5.1 : 1.0
      contains(dormiram) = True            False : True   =      5.1 : 1.0
          contains(saia) = True            False : True   =      5.1 : 1.0
   contains(tornariamos) = True            False : True   =      5.1 : 1.0
    contains(direitinho) = True            False : True   =      5.1 : 1.0
         contains(seios) = True            False : True   =      5.1 : 1.0
      contains(conectei) = True            False : True   =      5.1 : 1.0
       contains(virarao) = True            False : True   =      5.1 : 1.0
      contains(paisagem) = True            False : True   =      5.1 : 1.0
        contains(ocoreu) = True            False : True   =      5.1 : 1.0
contains(frequentadores) = True            False : True   =      5.1 : 1.0
       contains(magnata) = True            False : True   =      5.1 : 1.0
      contains(sinistro) = True            False : True   =      5.1 : 1.0
        contains(notase) 

    contains(literarias) = True            False : True   =      2.7 : 1.0
      contains(adequado) = True            False : True   =      2.7 : 1.0
        contains(maxima) = True            False : True   =      2.7 : 1.0
        contains(claros) = True            False : True   =      2.7 : 1.0
      contains(exemplos) = True            False : True   =      2.7 : 1.0
       contains(fluente) = True            False : True   =      2.7 : 1.0
        contains(quinto) = True            False : True   =      2.7 : 1.0
     contains(estaremos) = True            False : True   =      2.7 : 1.0
      contains(envolveu) = True            False : True   =      2.7 : 1.0
       contains(direita) = True            False : True   =      2.7 : 1.0
        contains(gemeos) = True            False : True   =      2.7 : 1.0
      contains(colegial) = True            False : True   =      2.7 : 1.0
        contains(indica) = True            False : True   =      2.7 : 1.0
         contains(senao) 

         contains(virar) = True            False : True   =      1.2 : 1.0
         contains(linha) = True            False : True   =      1.2 : 1.0
       contains(sabendo) = True            False : True   =      1.2 : 1.0
         contains(certo) = True            False : True   =      1.2 : 1.0
           contains(ler) = True            False : True   =      1.2 : 1.0
       contains(romance) = True            False : True   =      1.2 : 1.0
         contains(quase) = True            False : True   =      1.2 : 1.0
          contains(esas) = True             True : False  =      1.2 : 1.0
        contains(acabei) = True             True : False  =      1.1 : 1.0
           contains(sao) = True            False : True   =      1.1 : 1.0
        contains(tivese) = True            False : True   =      1.1 : 1.0
          contains(hora) = True            False : True   =      1.1 : 1.0
     contains(primeiras) = True            False : True   =      1.1 : 1.0
      contains(criancas) 

In [29]:
test_result = []
gold_result = []

for i in range(0,len(FVnP_test)):
    #test_result.append(classifier.classify(FVP_test[i])
    test_result.append(str(classifier.classify(FVnP_test[i])))
    gold_result.append('False')
    if test_result != gold_result:
        dict = {}
        dict = FVnP_test[i]
        list_key = list(dict.keys())
        #print("Classificação: " + 'True')
        #print("Palavras".center(70,"*"))
        for c in range(0,len(list_key)):
            if dict.get(list_key[c])==True:
                #print(list_key[c]) 
                1==1
                
        #print("Fim".center(70,"*"))

CM = nltk.ConfusionMatrix(gold_result, test_result)
print(CM)    

      |  F    |
      |  a  T |
      |  l  r |
      |  s  u |
      |  e  e |
------+-------+
False |<34>24 |
 True |  . <.>|
------+-------+
(row = reference; col = test)



In [69]:
labels = {'True', 'False'}

from collections import Counter
TP, FN, FP = Counter(), Counter(), Counter()
for i in labels:
    for j in labels:
        if i == j:
            TP[i] += int(CM[i,j])
        else:
            FN[i] += int(CM[i,j])
            FP[j] += int(CM[i,j])

print("label\tprecision\trecall\tf_measure")
for label in sorted(labels):
    precision, recall = 0, 0
    if TP[label] == 0:
        f_measure = 0
    else:
        precision = float(TP[label]) / (TP[label]+FP[label])
        recall = float(TP[label]) / (TP[label]+FN[label])
        f_measure = float(2) * (precision * recall) / (precision + recall)
    print(label+"\t"+str(precision)+"\t"+str(recall)+"\t"+str(f_measure))
    

label	precision	recall	f_measure
False	1.0	0.7962962962962963	0.8865979381443299
True	0	0	0
